<a href="https://colab.research.google.com/github/wmex91/Intelligent-Poka-Yoke/blob/main/DL_Thesis_Notebook_v4_3ClassesCrossValidation_Versione_in_consegna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CARICO GDRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#MODELLO DI ADDESTRAMENTO - YES CONFUSION MATRIX, NO DATA AUGMENTATION, YES TENSORBOARD 
#3 CLASSI, SI CROSS VALIDATION

#!pip install tensorboardcolab

from numpy.random import seed
#Random seed per Numpy
seed(1)

from PIL import Image
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras import optimizers
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
# from tensorflow import set_random_seed
# import keras.datasets
from IPython.display import display
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from skimage import io 
import random 
# from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
import time

In [ ]:
# FUNZIONI E METODI ############################################################

#Imposto una funzione per caricare i dati dalle directory di partenza
def loading_data(directory, sbatch, filter_type):
  for category in labels:  # itero tra OK, rOK e vOK

    path = os.path.join(directory, category)  # crea il percorso per OK, rOK e vOK
    class_num = labels.index(category)  # Imposta la classificazione 

    for img in tqdm(os.listdir(path)):  # itera ogni immagine per OK, rOK e vOK
          img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) # converte in array
          
          if filter_type == 0: 
            sbatch.append([img_array, class_num]) 
          
          if filter_type == 1: 
            filtered_image = gaussian_filter(img_array)
            sbatch.append([filtered_image, class_num])  # aggiunge a training_set; se faccio resize, aggiungo resize_array
          
          if filter_type == 2:
            filtered_image = filter_5x5(img_array)
            sbatch.append([filtered_image, class_num])  # aggiunge a training_set; se faccio resize, aggiungo resize_array
          
          if filter_type == 3:
            filtered_image = filter_3x3(img_array)
            sbatch.append([filtered_image, class_num]) 
          
          #resize_array = cv2.resize(img_array, (64, 64))  # resize 
          
          #plt.imshow(filtered_image, cmap='gray') # grafica
          #plt.show()  # mostra
          
          #break
    #break
    
#Funzione filtro passa alto piccolo 
def filter_3x3(data):
  kernel_3x3 = np.array([[-1, -1, -1],
                     [-1,  8, -1],
                     [-1, -1, -1]])
  highpass_3x3 = ndimage.convolve(data, kernel_3x3)

#Funzione filtro passa alto grande
def filter_5x5(data):
  kernel_5x5 = np.array([[-1, -1, -1, -1, -1],
                   [-1,  1,  2,  1, -1],
                   [-1,  2,  4,  2, -1],
                   [-1,  1,  2,  1, -1],
                   [-1, -1, -1, -1, -1]])    
  highpass_5x5 = ndimage.convolve(data, kernel_5x5)
  return(highpass_5x5)

#Fuzione filtro Gaussiano 
def gaussian_filter(data):
  gaussian_lowpass = ndimage.gaussian_filter(data, 3)
  gaussian_highpass = img_array - img_gaussian_lowpass
  return(gaussian_highpass)

################################################################################

In [ ]:
print(labels.index('OK'))
print(labels.index('rOK'))
print(labels.index('vOK'))

NameError: ignored

In [ ]:
import keras
# INIZIALIZZO GLI STRUMENTI IN USO NEL PROGRAMMA ###############################

#Pulisco la sessione dai dati settati in precedenza
K.clear_session()

#Random seed per Tensorflow
#set_random_seed(2)
tf.random.set_seed(2)
#Imposto una funzione per salvare il modello senza sovrascriverlo ai precedenti
nome = "MasterThesis-cnn-3classes-TB-nDA_" + str(time.localtime().tm_mday) + "." + str(time.localtime().tm_mon) + "." + str(time.localtime().tm_year) + "_"  + str(time.localtime().tm_hour+2) + ":"  + str(time.localtime().tm_min)

#Imposto l'oggetto tbc come TensorBoard per visualizzare i grafici di output 
#tbc = TensorBoardColab()

################################################################################

In [ ]:
# CARICAMENTO DATI #############################################################
data_dir = '/content/drive/My Drive/Colab Notebooks/Master-Thesis-Project/dataset/240x320/3_classes_K-FoldValidation'
          
#validation_data_dir = '/content/drive/My Drive/Colab Notebooks/dataset/240x320/2_classes/2 Classes_2 States/validation_set'

img_width, img_height = 320, 240
labels = ["OK", "rOK", "vOK"] # OK=dritto, rOK=rovescio, vOK=assente

#Preparo gli array per il caricamento dati
data_set = []

#Richiamo la funzione di caricamento dati
loading_data(data_dir, data_set, 2)

print("\nlunghezza data set: " + str(len(data_set)))

#Imposto un seed per lo shuffle e mescolo il data set in quanto i dati caricati sono ordinati per classe
random.seed(4)
random.shuffle(data_set)

################################################################################

100%|██████████| 201/201 [00:05<00:00, 33.64it/s]


lunghezza data set: 501


In [ ]:
# DIVISIONE DATA SET IN 5 SUBSET ###############################################

#Imposto 5 array di arrivo
sub_set1 = []
sub_set2 = []
sub_set3 = []
sub_set4 = []
sub_set5 = []

#LI DEVO MESCOLARE PRIMA DI INSERIRLI NEL T.SUBSET? O LI MESCOLO A OGNI ITERAZIONE K-ESIMA?

#Divido il dataset in 5 parti e assegno ogni parte al relativo subset
for i in range (len(data_set)):
  if i < 100: 
    sub_set1.append(data_set[i])
  elif i >= 100 and i < 200:
    sub_set2.append(data_set[i])
  elif i >= 200 and i < 300: 
    sub_set3.append(data_set[i])
  elif i >= 300 and i < 400: 
    sub_set4.append(data_set[i])
  elif i >= 400: 
    sub_set5.append(data_set[i])

print("lunghezza subset1: " + str(len(sub_set1)))
print("lunghezza subset2: " + str(len(sub_set2)))
print("lunghezza subset3: " + str(len(sub_set3)))
print("lunghezza subset4: " + str(len(sub_set4)))
print("lunghezza subset5: " + str(len(sub_set5)))
    
#Raccolgo tutti i subset in un array, per comodità 
t_subs = []
t_subs.append(sub_set1)
t_subs.append(sub_set2)
t_subs.append(sub_set3)
t_subs.append(sub_set4)
t_subs.append(sub_set5)

print("lunghezza t_subs: ")
print(len(t_subs))
print("")

################################################################################

NameError: ignored

In [ ]:
# INIZIALIZZAZIONE STRUTTURE DATI PER CROSS VALIDATION #########################

#Inizializzo l'array contente i risultati della Cross Validation
k_fold_conf_matrix = []
k_fold_accuracies = []
k_fold_precisions_ClassOK = []

################################################################################

# INIZIO CROSS VALIDATION ######################################################

for i in range(0, 5):
  #Inizia QUA il ciclo for per le 5 fasi di training
  #Inizializzo dei nuovi array per training, validation e test set; alla fine del ciclo, resetto i valori modificati
  training_set = []
  validation_set = []
  test_set = []

  #Assegno i subset generati ai vettore di training, validation e test a seconda del ciclo k a cui mi trovo 
  if i == 0: # test=0; validation=1
    for j in range(len(t_subs[0])):
      test_set.append(t_subs[0][j])
    for j in range(len(t_subs[1])):
      validation_set.append(t_subs[1][j])
    for j in range(len(t_subs[2])):
      training_set.append(t_subs[2][j])
    for j in range(len(t_subs[3])):
      training_set.append(t_subs[3][j])
    for j in range(len(t_subs[4])):
      training_set.append(t_subs[4][j])
        
  if i == 1: #test=1; validation=2
    for j in range(len(t_subs[1])):
      test_set.append(t_subs[1][j])
    for j in range(len(t_subs[2])):
      validation_set.append(t_subs[2][j])
    for j in range(len(t_subs[3])):
      training_set.append(t_subs[3][j])
    for j in range(len(t_subs[4])):
      training_set.append(t_subs[4][j])
    for j in range(len(t_subs[0])):
      training_set.append(t_subs[0][j])    
  
  if i == 2: #test=2; validation=3
    for j in range(len(t_subs[2])):
      test_set.append(t_subs[2][j])
    for j in range(len(t_subs[3])):
      validation_set.append(t_subs[3][j])
    for j in range(len(t_subs[4])):
      training_set.append(t_subs[4][j])
    for j in range(len(t_subs[0])):
      training_set.append(t_subs[0][j])
    for j in range(len(t_subs[1])):
      training_set.append(t_subs[1][j])
  
  if i == 3: #test=3; validation=4
    for j in range(len(t_subs[3])):
      test_set.append(t_subs[3][j])
    for j in range(len(t_subs[4])):
      validation_set.append(t_subs[4][j])
    for j in range(len(t_subs[0])):
      training_set.append(t_subs[0][j])
    for j in range(len(t_subs[1])):
      training_set.append(t_subs[1][j])
    for j in range(len(t_subs[2])):
      training_set.append(t_subs[2][j])
  
  if i == 4: #test=4; validation=0
    for j in range(len(t_subs[4])):
      test_set.append(t_subs[4][j])
    for j in range(len(t_subs[0])):
      validation_set.append(t_subs[0][j])
    for j in range(len(t_subs[1])):
      training_set.append(t_subs[1][j])
    for j in range(len(t_subs[2])):
      training_set.append(t_subs[2][j])
    for j in range(len(t_subs[3])):
      training_set.append(t_subs[3][j])

  print("lunghezza training, validation e test set")
  print(len(training_set))
  print(len(validation_set))
  print(len(test_set))
        
  #Preprocessing training set 
  #Popolo il vettore delle train feature e delle train target
  X_train = []
  y_train = []

  for features, label in training_set:
      X_train.append(features)
      y_train.append(label)

  X_train = np.array(X_train).reshape(-1, img_width, img_height, 1)

  #Normalizzazione dei dati 
  X_train = X_train/255.0

  #Imposto il vettore dei target in 3 categorie
  y_train_3 = to_categorical(y_train, 3)

  #Preprocessing validation set
  #Popolo il vettore delle validaiton feature e delle validation target
  X_validation = []
  y_validation = []

  for features,label in validation_set:
      X_validation.append(features)
      y_validation.append(label)

  X_validation = np.array(X_validation).reshape(-1, img_width, img_height, 1)

  # Normalizzazione dei dati 
  X_validaiton = X_validation/255.0
  
  #Imposto il vettore dei target in 3 categorie
  y_validation_3 = to_categorical(y_validation, 3)
  
  #Preprocessing test set
  # Popolo il vettore delle test feature e delle test target
  X_test = []
  y_test = []

  for features,label in test_set:
      X_test.append(features)
      y_test.append(label)

  X_test = np.array(X_test).reshape(-1, img_width, img_height, 1)

  # Normalizzazione dei dati 
  X_test = X_test/255.0

  # Imposto il vettore dei target in 3 categorie
  y_test_3 = to_categorical(y_test, 3)
  
  print("Dimensione validation set dopo labeling: " + str(len(y_validation_3)))
  print("")
        
  print("K-Fold Cross Validation con k = " + str(i))

  ##############################################################################
  
  # INIZIO MODELLO RETE NEURALE ################################################
  #Pulizia da modelli precedentemente addestrati
  K.clear_session()
  
  # Initialize the CNN
  classifier = Sequential()

  # Step 1 - Convolution - Commentato lo strato standard
  classifier.add(Convolution2D(filters=32, kernel_size=3, input_shape=X_train.shape[1:], padding='same', activation='relu'))

  # Step 2 - Pooling
  classifier.add(MaxPooling2D(pool_size=(2, 2)))

  # Step 3 - Flattening - Srotolo la matrice in output ai filtri come un'unico vettore
  classifier.add(Flatten())

  classifier.add(Dense(64, activation='relu'))

  # Step 4 - Strato di Dropout - Per ridurre l'overfitting; da testare in seguito
  classifier.add(Dropout(0.5))

  # Step 5 - Imposto l'output layer
  classifier.add(Dense(3, activation='softmax'))

  # Imposto esplicitamente l'ottimizzatore per alzare il learning rate
  # optimizer_set = optimizers.Adam(lr=0.1)

  # Step 6 - Compilazione della CNN
  classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Step 7 - Implemento l'early stopping per diminuire l'overfitting
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

  # Step 8 - Addestramento della CNN
  classifier.fit(X_train, y_train_3, batch_size=32, epochs=500, validation_data=(X_validation, y_validation_3), callbacks=[es])

  # Salva il modello; posso salvare i pesi (.model extension) o l'intero modello (.h5 extension)
  # Per richiamare il modello salvato: new_model = keras.models.load_model('my_model.h5')
  # Vedi in => https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb#scrollTo=5NDMO_7kS6Do
  # classifier.save_weights('/content/drive/My Drive/Colab Notebooks/PhD_Notebooks/Master-Thesis-Project/dataset/model_saved/' + nome + '.h5')

  ################################################################################

  # FASE DI PREDIZIONE E VALUTAZIONE PRESTAZIONI #################################

  #Effettuo la predizione del test set 
  predizione_no_cutoff = classifier.predict(X_test)

  # Converto i valori predetti in virgola in interi: 0 non predetto, 1 predetto. La posizione nel vettore indica lo stato predetto
  predizione_cutoff_no_labels = []

  for index in range(len(predizione_no_cutoff)): 
    min = 0
    z = -1
    for value in range(0,3): 
      if predizione_no_cutoff[index][value] > min:
        min = predizione_no_cutoff[index][value]
        z = value
    #test_risultato[index][z] = 1

    conversione = []
    for valuetest in range(0,3): 
      if valuetest == z:
        conversione.append(1)
      else: 
        conversione.append(0)

    predizione_cutoff_no_labels.append(conversione)

  # Converto i valori predetti da binari in valori di labels: stato 1 = 0, stato 2 = 1, stato 3 = 2
  predizione_labels = []
  for index1 in range(len(y_test_3)):
    x = -1
    for index2 in range(0,3):
      if predizione_cutoff_no_labels[index1][index2] == 1: 
        x = index2
    predizione_labels.append(x)
  
  # Converto i valori di test in posizione in valori di labels: stato 1 = 0, stato 2 = 1, stato 3 = 2
  y_test_3_labels = []

  for index1 in range(len(y_test_3)):
    y = -1
    for index2 in range(0,3):
      if y_test_3[index1][index2] == 1: 
        y = index2
    y_test_3_labels.append(y)

  #Converto gli array predetti ed esatti in numpy.array per la matrice di confusione di Scikit Learn
  y_test_3_labels = np.array(y_test_3_labels)
  predizione_labels = np.array(predizione_labels)

  if i == 0: 
    # Definisco la matrice di confusione per k-fold=0
    confusion_matrix0 = confusion_matrix(y_test_3_labels, predizione_labels, labels=[0, 1, 2])
    # Aggiungo la matrice di confusione al vettore k_fold_conf_matrix
    k_fold_conf_matrix.append(confusion_matrix0)
    # Stampo la matrice di confusione
    print("Matrice di confusione")
    print(confusion_matrix0)
    # Computo accuratezza, precisione OK/rOK, false_positive_rate rOK
    accuratezza = (confusion_matrix0[0][0] + confusion_matrix0[1][1] + confusion_matrix0[2][2])/(len(y_test_3_labels))
    precisione_OK = (confusion_matrix0[0][0])/(confusion_matrix0[0][0] + confusion_matrix0[0][1] + confusion_matrix0[0][2])
      
    false_pos_rate_rOK = (confusion_matrix0[1][0] + confusion_matrix0[1][2])/(confusion_matrix0[1][0] + confusion_matrix0[1][2] + confusion_matrix0[0][0] + confusion_matrix0[2][2])
    # Appendo le 3 metriche alle relative liste
    k_fold_accuracies.append(accuratezza)
    k_fold_precisions_ClassOK.append(precisione_OK)
    # Stampo le metriche
    print("Accurezza a k = " + str(i) + ":" + str(accuratezza))
    print("Precisione classe OK a k = " + str(i) + ": " + str(k_fold_precisions_ClassOK))
    print("")
    
  if i == 1: 
    confusion_matrix1 = confusion_matrix(y_test_3_labels, predizione_labels, labels=[0, 1, 2])
    k_fold_conf_matrix.append(confusion_matrix1)
    print("Matrice di confusione")
    print(confusion_matrix1)
    accuratezza = (confusion_matrix1[0][0] + confusion_matrix1[1][1] + confusion_matrix1[2][2])/(len(y_test_3_labels))
    precisione_OK = (confusion_matrix1[0][0])/(confusion_matrix1[0][0] + confusion_matrix1[0][1] + confusion_matrix1[0][2])
    k_fold_accuracies.append(accuratezza)
    k_fold_precisions_ClassOK.append(precisione_OK)
    print("Accurezza a k = " + str(i) + ": " + str(accuratezza))
    print("Precisione classe OK a k = " + str(i) + ": " + str(k_fold_precisions_ClassOK))
    print("")

  if i == 2: 
    confusion_matrix2 = confusion_matrix(y_test_3_labels, predizione_labels, labels=[0, 1, 2])
    k_fold_conf_matrix.append(confusion_matrix2)
    print("Matrice di confusione")
    print(confusion_matrix2)
    accuratezza = (confusion_matrix2[0][0] + confusion_matrix2[1][1] + confusion_matrix2[2][2])/(len(y_test_3_labels))
    precisione_OK = (confusion_matrix2[0][0])/(confusion_matrix2[0][0] + confusion_matrix2[0][1] + confusion_matrix2[0][2])
    k_fold_accuracies.append(accuratezza)
    k_fold_precisions_ClassOK.append(precisione_OK)
    print("Accurezza a k = " + str(i) + ": " + str(accuratezza))
    print("Precisione classe OK a k = " + str(i) + ": " + str(k_fold_precisions_ClassOK))
    print("")
    print("Vettore y_train")
    print(y_train_3)
    print("Vettore y_validation")
    print(y_validation_3)
    print("Vettore y_test")
    print(y_test_3)
    print("")

  if i == 3: 
    confusion_matrix3 = confusion_matrix(y_test_3_labels, predizione_labels, labels=[0, 1, 2])
    k_fold_conf_matrix.append(confusion_matrix3)
    print("Matrice di confusione")
    print(confusion_matrix3)
    accuratezza = (confusion_matrix3[0][0] + confusion_matrix3[1][1] + confusion_matrix3[2][2])/(len(y_test_3_labels))
    precisione_OK = (confusion_matrix3[0][0])/(confusion_matrix3[0][0] + confusion_matrix3[0][1] + confusion_matrix3[0][2])
    k_fold_accuracies.append(accuratezza)
    k_fold_precisions_ClassOK.append(precisione_OK)
    print("Accurezza a k = " + str(i) + ": " + str(accuratezza))
    print("Precisione classe OK a k = " + str(i) + ": " + str(k_fold_precisions_ClassOK))
    print("")

  if i == 4: 
    confusion_matrix4 = confusion_matrix(y_test_3_labels, predizione_labels, labels=[0, 1, 2])
    k_fold_conf_matrix.append(confusion_matrix4)
    print("Matrice di confusione")
    print(confusion_matrix4)
    accuratezza = (confusion_matrix4[0][0] + confusion_matrix4[1][1] + confusion_matrix4[2][2])/(len(y_test_3_labels))
    precisione_OK = (confusion_matrix4[0][0])/(confusion_matrix4[0][0] + confusion_matrix4[0][1] + confusion_matrix4[0][2])
    k_fold_accuracies.append(accuratezza)
    k_fold_precisions_ClassOK.append(precisione_OK)
    print("Accurezza a k = " + str(i) + ": " + str(accuratezza))
    print("Precisione classe OK a k = " + str(i) + ": " + str(k_fold_precisions_ClassOK))
    print("")
    
# FINE CROSS VALIDATION ########################################################


NameError: ignored

In [ ]:
print(predizione_cutoff_no_labels)

[[1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0]

In [ ]:
print("Matrice di confusione con k = 0")
print(k_fold_conf_matrix[0])
print("Accuratezza 'K=0': " + str(k_fold_accuracies[0]))
print("Precisione 'K=0': " + str(k_fold_precisions_ClassOK[0]))
print("")

print("Matrice di confusione con k = 1")
print(k_fold_conf_matrix[1])
print("Accuratezza 'K=1': " + str(k_fold_accuracies[1]))
print("Precisione 'K=1': " + str(k_fold_precisions_ClassOK[1]))
print("")

print("Matrice di confusione con k = 2")
print(k_fold_conf_matrix[2])
print("Accuratezza 'K=2': " + str(k_fold_accuracies[2]))
print("Precisione 'K=2': " + str(k_fold_precisions_ClassOK[2]))
print("")

print("Matrice di confusione con k = 3")
print(k_fold_conf_matrix[3])
print("Accuratezza 'K=3': " + str(k_fold_accuracies[3]))
print("Precisione 'K=3': " + str(k_fold_precisions_ClassOK[3]))
print("")

print("Matrice di confusione con k = 4")
print(k_fold_conf_matrix[4])
print("Accuratezza 'K=4': " + str(k_fold_accuracies[4]))
print("Precisione 'K=4': " + str(k_fold_precisions_ClassOK[4]))
print("")

accuratezza_macro = (k_fold_accuracies[0] + k_fold_accuracies[1] + k_fold_accuracies[2] + k_fold_accuracies[3] + k_fold_accuracies[4])/5
precisione_macro = (k_fold_precisions_ClassOK[0] + k_fold_precisions_ClassOK[1] + k_fold_precisions_ClassOK[2] + k_fold_precisions_ClassOK[3] + k_fold_precisions_ClassOK[4])/5

micro_conf_matrix = []
micro_conf_matrix = k_fold_conf_matrix[0] + k_fold_conf_matrix[1] + k_fold_conf_matrix[2] + k_fold_conf_matrix[3] + k_fold_conf_matrix[4]

denominatore_micro = 0

for i in range (0,3):
  for j in range(0,3): 
    denominatore_micro = denominatore_micro + micro_conf_matrix[i][j]

accuratezza_micro = (micro_conf_matrix[0][0] + micro_conf_matrix[1][1] + micro_conf_matrix[2][2])/denominatore_micro
precisione_micro = (micro_conf_matrix[0][0])/(micro_conf_matrix[0][0] + micro_conf_matrix[0][1] + micro_conf_matrix[0][2])

print("Accuratezza macro: " + str(accuratezza_macro))
print("Accuratezza micro: " + str(accuratezza_micro))
print("")
print("Precisione OK macro: " + str(precisione_macro))
print("Precisione OK micro: " + str(precisione_micro))


In [ ]:
# Calcolo precisione per classe rOK 
# Calcolo il bilancio dei falsi positivi. Il sistema può essere tollerante a un 
# paraolio dritto classificato come rovescio ma non il contrario.

# K-Fold = 0
precisione_rOK0 = (confusion_matrix0[1][1])/(confusion_matrix0[1][0] + confusion_matrix0[1][2])
false_pos_rate_rOK0 = (confusion_matrix0[1][0] + confusion_matrix0[1][2])/(confusion_matrix0[1][0] + confusion_matrix0[1][2] + confusion_matrix0[0][0] + confusion_matrix0[2][2])
print('Matrice di confusione con k = 0')
print(confusion_matrix0)
print('Precisione rOK k = 0: ', precisione_rOK)
print('False Positive rate k = 0: ', false_pos_rate_rOK)

# K-Fold = 1
precisione_rOK1 = (confusion_matrix1[1][1])/(confusion_matrix1[1][0] + confusion_matrix1[1][2])
false_pos_rate_rOK1 = (confusion_matrix1[1][0] + confusion_matrix1[1][2])/(confusion_matrix1[1][0] + confusion_matrix1[1][2] + confusion_matrix1[0][0] + confusion_matrix1[2][2])
print('Matrice di confusione con k = 1')
print(confusion_matrix1)
print('Precisione rOK k = 1: ', precisione_rOK1)
print('False Positive rate k = 1: ', false_pos_rate_rOK1)

# K-Fold = 2
precisione_rOK2 = (confusion_matrix2[1][1])/(confusion_matrix2[1][0] + confusion_matrix2[1][2])
false_pos_rate_rOK2 = (confusion_matrix2[1][0] + confusion_matrix2[1][2])/(confusion_matrix2[1][0] + confusion_matrix2[1][2] + confusion_matrix2[0][0] + confusion_matrix2[2][2])
print('Matrice di confusione k = 2')
print(confusion_matrix2)
print('Precisione rOK k = 2: ',precisione_rOK2)
print('False Positive rate k = 2: ', false_pos_rate_rOK2)

# K-Fold = 3
precisione_rOK3 = (confusion_matrix3[1][1])/(confusion_matrix3[1][0] + confusion_matrix3[1][2])
false_pos_rate_rOK3 = (confusion_matrix3[1][0] + confusion_matrix3[1][2])/(confusion_matrix3[1][0] + confusion_matrix3[1][2] + confusion_matrix3[0][0] + confusion_matrix3[2][2])
print('Matrice di confusione k = 3')
print(confusion_matrix3)
print('Precisione rOK k = 3: ',precisione_rOK3)
print('False Positive rate k = 3: ', false_pos_rate_rOK3)

# K-Fold = 4
precisione_rOK4 = (confusion_matrix4[1][1])/(confusion_matrix4[1][0] + confusion_matrix4[1][2])
false_pos_rate_rOK4 = (confusion_matrix4[1][0] + confusion_matrix4[1][2])/(confusion_matrix4[1][0] + confusion_matrix4[1][2] + confusion_matrix4[0][0] + confusion_matrix4[2][2])
print('Matrice di confusione k = 4')
print(confusion_matrix4)
print('Precisione rOK k = 4: ',precisione_rOK4)
print('False Positive rate k = 4: ', false_pos_rate_rOK4)

Matrice di confusione con k = 0
[[24  0  3]
 [ 0 30  0]
 [ 1  0 42]]
Precisione rOK k = 0:  inf
False Positive rate k = 0:  0.0
Matrice di confusione con k = 1
[[27  0  0]
 [ 0 29  0]
 [ 0  0 44]]
Precisione rOK k = 1:  inf
False Positive rate k = 1:  0.0
Matrice di confusione k = 2
[[ 0  0 28]
 [ 0  0 36]
 [ 0  0 36]]
Precisione rOK k = 2:  0.0
False Positive rate k = 2:  0.5
Matrice di confusione k = 3
[[30  0  1]
 [ 0 26  0]
 [ 0  0 43]]
Precisione rOK k = 3:  inf
False Positive rate k = 3:  0.0
Matrice di confusione k = 4
[[36  0  1]
 [ 0 29  0]
 [ 0  0 35]]
Precisione rOK k = 4:  inf
False Positive rate k = 4:  0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in long_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in long_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in long_scalars
